<style type="text/css">
.time_spent {
    width: 3em;
    border-style: none;
    background-color: silver;
    font-weight: bold;
    padding-left: 5px;
}
</style>

# [WS13/14] Mathematics for Robotics and Control Assignment 005 - SVD decomposition and Jacobians

In [ ]:
#TEAM SAI KIRAN AND SABEEH

In [1]:
try:
    shell = get_ipython()
    shell.enable_pylab("inline")
except NameError:
    pass

import numpy
import matplotlib
import scipy.linalg 
from matplotlib import pylab, mlab, pyplot
np = numpy
plt = pyplot
            
from IPython.display import display
from IPython.core.pylabtools import figsize, getfigs
import IPython
from pylab import *
from numpy import *
from sympy import cos, sin
from sympy import *


---

### SVD decomposition

Consider the following scenario description: the robot (the blue sphere in the picture below) moves in a rectangular arena surrounded by four pillars at every corner. On top of each of the pillars, a spherical object is mounted that sends some form of signal that the robot is able to receive.

In [5]:
IPython.core.display.Image(r"images/scene001.png", width=640, embed=True)

Based on some physical characteristica of the signal, the robot is able to deduce it's distance to the center of each of the spheres (the length of the dotted red lines). Examples could be the signal strength of a WiFi signal, or the time an ultrasonic signal sent by each of the spheres took to be received by the robot. Here are some properties of the arena:

In [6]:
IPython.core.display.Image(r"images/g7142.png", width=640, embed=True)

The robot moves around in the arena, constantly receiving the distance to each of the spheres as a quadruple. Looking at the picture above, the elements of the tuple are returned in a counter-clockwise manner, i.e. the pillar in the lower left corner will be returned at the first position inside the tuple, the distance to the sphere on top of the pillar in the lower right corner will be at the second position of the tuple and so on. Now, one of your colleagues has written the framework for the robot and wants you to verify that it's working. She hands you a bunch of test data and asks you to determine the position of the robot while it is moving around the arena. 

* **Implement a method to determine the robot's position via the distance to each of the spheres.**
* **Write an update_position() method using the signature shown below. Do NOT modify the function's signature and comment in the assert before testing your implementation.**
* **Return the robot's position from the update_position method as an iterable containing the X, Y and Z position of the robot**
* **Assume that the position of the center of the sphere in the lower left corner is (0, 0, 0)**

Remember what we discussed at the end of the last lab class, especially think of how a point that you only know the distance of can be located anywhere on the surface of a sphere whose radius equals said distance. Also remember that the position of each of the signal emitters (green spheres) is known. It might also help to think about how many of the four available signals you need in order to determine the position of the robot. Additionally, is there any "hidden" information available to you that imposes further constraints to the equations you are supposed to set up to determine the robot's position?

In [2]:
def update_position(distances):
    # Determine robot position
    # ...
    # <your code here>
    # ...
    # assert len(position) == 3
    # return position
    dx = 6
    dy = 4.5
    dz = 5.3
    x = Symbol('x')
    y = Symbol('y')
    z = Symbol('z')
    eq1 = x**2+y**2+(z-dz)**2 - distances[0]
    eq2 = (x-dx)**2+y**2+(z-dz)**2 - distances[1]
    eq3 = (x-dx)**2+(y-dy)**2+(z-dz)**2 - distances[2]
    eq4 = x**2+(y-dy)**2+(z-dz)**2 - distances[3]
    fofx_mat = Matrix([[eq1],[eq2],[eq3]])
    syms = Matrix([x,y,z])
    fdashx_mat = fofx_mat.jacobian(syms)
    num = fofx_mat.subs([(x,0),(y,0),(z,0)])
    den = fdashx_mat.subs([(x,0),(y,0),(z,0)])
    inverseden = np.linalg.inv(den)
    mul = inverseden* num
    x0 = Matrix([[0],[0],[0]])
    fofx_ini = x0 - mul
    print fofx_ini
    iterations = 10
    for i in range(iterations):
        num = fofx_mat.subs([(x,fofx_ini[0]),(y,fofx_ini[1]),(z,fofx_ini[2])])
        den = fdashx_mat.subs([(x,fofx_ini[0]),(y,fofx_ini[1]),(z,fofx_ini[2])])
        inverseden = np.linalg.inv(den)
        mul = inverseden* num
        fofx_next = fofx_ini-mul
        print fofx_next
        fofx_ini = fofx_next
    pos = (fofx_next[0],fofx_next[1],fofx_next[2])
    return pos
    raise NotImplementedError()

### Jacobians

*This assignment will require some reading. Brace yourself. [Here](http://excelsior.cs.ucsb.edu/courses/cs290n_cg_modeling/notes/uiuc_notes/chap5.pdf) is a chapter which you may consider as an introduction to the topic.*

The following picture describes a robot arm with three rotational joints and two links:

In [3]:
IPython.core.display.Image(r"images/robot_arm001.png")

The left part of the above figure shows four different frames attached to the arm of the robot, frame 0, 1, 2 and frame 3. $X_0$ and $Y_0$ describe the base frame, $Y_1$ is the unit vector pointing to the direction of the first link's $Y$ axis. Accordingly, $X_2$ denotes the direction of the second link's $X$ axis. Finally, $X_3$ and $Y_3$ specify the orientation of the end effector frame, which you can assume to be fixed.

1. **Specify the transformation matrices between frame 0 and frame 1, between frame 1 and frame 2 and finally the transformation from frame 2 to frame 3. Yes, this is a plain and simple repetition of what you learned in your very first lecture!**

Assuming angular velocities are expressed using a common frame, they can be added as vectors. Now, given link number $i$, it's velocity will be whatever the velocity of link $i-1$ was, plus whatever velocities were added by joint $i$. Another way to think of it is: before you can add the angular velocities, you have to transform one of the velocities to the coordinate frame of the other.

2. **Specify the term to compute the angular velocity $\omega_i$ in terms of $\omega_{i-1}$ with respect to frame $i-1$. Remember that angular velocity is the derivative of a joint value $\theta$ with respect to a rotation axis.**

3. **Specify the Jacobian matrix that relates the velocity of the end effector relative to the joint angles of the robotic arm!**

4. **Does the Jacobian have an inverse? If so, is the matrix *always* invertible? What is the physical meaning of the matrix inverse?**

5. **Does the Jacobian always have full rank? How does the rank of the Jacobian affect the velocity mapping?**

In [3]:
#1. Transformations matrices mentioned below
theta1 = Symbol('theta1')
theta2 = Symbol('theta2')
L1 = Symbol('L1')
L2 = Symbol('L2')

In [4]:
T1to0 = np.array([(cos(theta1), -sin(theta1), 0, 0), (sin(theta1), cos(theta1), 0, 0), (0, 0, 1, 0),(0, 0, 0, 1)])
T2to1 = np.array([(cos(theta2), -sin(theta2), 0, L1), (sin(theta2), cos(theta2), 0,0), (0, 0, 1, 0), (0, 0, 0, 1)])
T3to2 = np.array([(1, 0, 0, L2), (0, 1, 0, 0), (0, 0, 1, 0), (0, 0, 0, 1)])
print T1to0 #transformation matrix between frame 0 and frame 1
print T2to1 #transfomation matrix between frame 1 and frame 2
print T3to2 #trasnformation matrix between frame 2 and frame 3

[[cos(theta1) -sin(theta1) 0 0]
 [sin(theta1) cos(theta1) 0 0]
 [0 0 1 0]
 [0 0 0 1]]
[[cos(theta2) -sin(theta2) 0 L1]
 [sin(theta2) cos(theta2) 0 0]
 [0 0 1 0]
 [0 0 0 1]]
[[1 0 0 L2]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]]


In [5]:
#angular velocity W_i in terms of W_i-1 
print ("1. W_i = R * W_i-1 (where R is the Rotation Matrix)") 

1. W_i = R * W_i-1 (where R is the Rotation Matrix)


In [6]:
Jacobian = np.array([(-L1*sin(theta1)-L2 * sin(theta1 + theta2), -L2 * sin(theta1 + theta2)), (L1 * cos(theta1) + L2 * cos(theta1 + theta2), L2 * cos(theta1 + theta2))])
print Jacobian
print Jacobian.shape

[[-L1*sin(theta1) - L2*sin(theta1 + theta2) -L2*sin(theta1 + theta2)]
 [L1*cos(theta1) + L2*cos(theta1 + theta2) L2*cos(theta1 + theta2)]]
(2, 2)


In [7]:
print ("3. The Inverse of the Jacobian exists if the respective matrix is non-singular, but if it is singular inverse doesn't exist. In this sense if the inverse of the marix doesn't exists then it infers that the point in the configuration space is unreachable or beyond reach.")

3. The Inverse of the Jacobian exists if the respective matrix is non-singular, but if it is singular inverse doesn't exist. In this sense if the inverse of the marix doesn't exists then it infers that the point in the configuration space is unreachable or beyond reach.


In [8]:
print ("4. It is not mandatory that a Jacobian must have full rank. If it doesn't have a full rank it infers that it is not attainable direction with instantaneous motion.")

4. It is not mandatory that a Jacobian must have full rank. If it doesn't have a full rank it infers that it is not attainable direction with instantaneous motion.
